## Intro
Taking a first look at the [data](https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Daily-Totals/5neh-572f), it looks like we have a station ID and name, and number of rides. What is notably absent: *which* train people board (many stations serve multiple lines), locations for the stations, and number of unique riders (or unique IDs for each rider to deduce that). CTA doesn't make you "tap out" when you exit the train, so you don't know where individual rides end, and you don't know how many people are passing through stations as they *leave* trains.   


## Preprocessing 

I started out by putting the dataset into a table in a PostgreSQL database. My favorite way to explore data is using SQL - I find the grouping functions are more intuitive, plus aggregate/apply functions are very fast on larger datasets. Plus it's good practice to use a database for reproducibility and collaboration.

```
select count(*) 
from cta.daily_ridership_by_station;

select min(date) 
from cta.daily_ridership_by_station;
select max(date) 
from cta.daily_ridership_by_station; 

select count(distinct station_id) 
from cta.daily_ridership_by_station;
select count(distinct stationname) 
from cta.daily_ridership_by_station;
```

As of the writing, the data contains 861,886 records, from January 1, 2001 to July 31, 2017. There are 147 different station names with 146 different station IDs. 

Intuitively, it seems that `station_id` and `stationname` should always be a unique pair (i.e. have a 1-1 relationship), but this isn't the case. While it is *is* true that no station *name* has more than one corresponding ID:
```
select stationname, count(distinct station_id) 
from cta.daily_ridership_by_station 
group by 1 
order by 2 desc;

       stationname        | count 
--------------------------+-------
 18th                     |     1
 35-Bronzeville-IIT       |     1
 35th/Archer              |     1
 43rd                     |     1
 47th-Dan Ryan            |     1
```

We find the one where this doesn't hold true: 
```
select station_id, count(distinct stationname) 
from cta.daily_ridership_by_station 
group by 1 
order by 2 desc;

station_id | count 
------------+-------
      40140 |     2
      40020 |     1
      
      
select station_id, stationname 
from cta.daily_ridership_by_station 
where station_id = '40140' 
group by 1,2;

 station_id |   stationname   
------------+-----------------
      40140 | Skokie
      40140 | Dempster-Skokie
```
Looks like the offending station is "Skokie" or "Dempster-Skokie." 
The next question is: is this really the same station? To get that answer, we can look at the CTA map (are these two separate locations?). Another way to find out is to look in the data:
```
select max(date) 
from cta.daily_ridership_by_station 
where station_id = '40140' 
and stationname = 'Skokie';

    max     
------------
 2012-03-31

select min(date) 
from cta.daily_ridership_by_station 
where station_id = '40140' 
and stationname = 'Dempster-Skokie';

min     
------------
 2012-04-01

```
It looks like all the observations using the name "Skokie" are before April 2012, and after April 2012 they started calling the station "Dempster-Skokie." This means we should probably treat them as one entity. (*This wouldn't have caused a problem if we index only by `station_id`, but it is worth noting.*) 

Next I think it's worth looking at the number of observations for each station (days on which there are counts for a given station):

```
select station_id, count(distinct date) 
from cta.daily_ridership_by_station 
group by 1 
order by 2 desc;
```

We see that most have 6056 days (which makes sense for the interval from January 1, 2001 to July 31, 2017). A handful have 6026 (missing about one month), and then a few errant stations have fewer - 5905, 2953, 1948, 1918, 943, 31. 

The lowest count (31) is for the Homan stop, which according to Wikipedia was [open from 1894 to 1994](https://en.wikipedia.org/wiki/Homan_station_(CTA_Green_Line)) and hasn't reopened since then (spooky!!). This station only has observations for the month of July 2001. There's a chance this could be a coding error in human entry - since it was operating in 1901, it could be they had observations with 01 in the date and accidentally labeled it for 2001. This happens pretty often. There are no other stops named Homan. There IS a Kedzie-Homan-Forest Park (40250), and a Conservatory stop, which is where that was moved to, but those both have complete sets of observations for July 2001, which raises questions about this stop as well as the overall data quality. This is something worth being aware of, and if for some reason this showed up as the highest average or SD we'd be suspicious of the results, but if it doesn't have the highest average or weekday SD, it's not affecting the results, so we can leave it in. 

The second least-frequently-observed station is Cermak-McCormick Place, which was opened in 2015, so it makes sense that it has about 1/6 of the observations that the others do. Similarly, the Morgan station on the Green Line was closed and reopened in 2012.  

```
select station_id, date, count(*) 
from cta.daily_ridership_by_station 
group by 1,2 
having count(*) > 1
order by 3 desc;

 station_id |    date    | count 
------------+------------+-------
      40450 | 2011-08-10 |     2
      40600 | 2011-07-01 |     2
      41200 | 2011-07-29 |     2
      40710 | 2011-07-30 |     2
      40580 | 2011-07-20 |     2
      40350 | 2011-07-11 |     2
      40220 | 2011-08-02 |     2
```
I'd also like to make sure that no stations have multiple counts on one day, but it looks like there are a handful that do have this problem. They all seem to be in July or August of 2011, and there are at most 2 observations for a given station for a given date. We check a few of them out more in-depth:

```
select * from cta.daily_ridership_by_station 
where station_id = '40580' 
and date = '2011-07-03';

 station_id | stationname |    date    | daytype | rides 
------------+-------------+------------+---------+-------
      40580 | 54th/Cermak | 2011-07-03 | U       |  1284
      40580 | 54th/Cermak | 2011-07-03 | U       |  1285
```      
For example, for this one, there are a handful of days in July of 2011 where 2 numbers are recorded, and the numbers are almost always really similar. So for the overall average it might make sense to leave it and consider those "individual days" because adding one more observation that's really close doesn't affect it too much. It probably doesn't make sense to add them together, because they don't seem to be two observations meant to add up. In these cases, the optimal solution is to get explanation/confirmation from the client or project partner, but in the case they're not available, we usually just try to do something that "makes sense," as well as document any assumptions we made and the potential effects on the results if those assumptions don't hold. As the data people, we can decide to average the numbers for a given day, or throw out one of the observations, or add them together, but it's possible that that is totally nonsensical in the context of the data generating mechanism. 


# Warmup Question 1 
*Note that average and standard deviation are taken across all days with observations*
```
select station_id, stationname, avg(rides) 
from cta.daily_ridership_by_station 
group by 1,2 
order by 3 desc;

 station_id |       stationname        |          avg           
------------+--------------------------+------------------------
      40380 | Clark/Lake               |     13760.167244510484
      41660 | Lake/State               |     13490.032518983163
      41450 | Chicago/State            |     12956.923877146631

``` 
For the (naive) average, we can just average by `station_id` over all observations. We find that the highest number of rides is station 40380, with a mean of about 13760 rides per observation, which is Clark & Lake. Anyone who has lived in Chicago could probably have told you that (without even using data) - it's the most heavily traveled station, in the middle of the Loop, and serves several train lines. The next-highest include State & Lake, which also makes sense.

A potential "to-do" would be to adjust this for the multiple observations at a given station. 

# Warmup Question 2 
```
with weekday_rides as (
select station_id, stationname, rides
from cta.daily_ridership_by_station
where daytype = 'W')
select station_id, stationname, stddev(rides)
from weekday_rides 
group by 1,2
order by 3 desc;


station_id |       stationname        |       stddev        
------------+--------------------------+---------------------
      41660 | Lake/State               |   4356.923361286698
      40500 | Washington/State         |   3890.640502656217
      41420 | Addison-North Main       |   3320.485144957506

```
The highest standard deviations are at State & Lake and State & Washington. This is interesting because they are basically the same station - the Red Line moved to its own station at State & Lake, so Washington is closed on the Red Line. They are followed by Addison (the stop at Wrigley Field), which makes sense .  

'Weekdays' are denoted in the data by a 'W' in the `day` field, whereas Saturdays are denoted by an 'A', and Sundays and Holidays (New Year's Day, Memorial Day, Independence Day, Labor Day, Thanksgiving, and Christmas Day) are purportedly denoted by a 'U'. However, it's worth checking that they are actually labeled as such, especially since I noticed on the portal some of the data looked like it was January 1 but had a 'W'. However, in the data that didn't actually hold.



# Challenge Question

> Imagine you’re a business owner in Chicago looking to open a new location. Any kind of business will do. In the form of writing, potentially supplemented by sketches (computer-drawn or hand-drawn) and links, we want to see your response to these questions:
- What questions could you potentially explore/answer with this data?
- Ideally, what other data would you gather or combine to learn even more?
- How would you want to see data presented, to make it actionable by you or others?

-With the original dataset, you can look at where incoming traffic is concentrated, by station and over time - where there are the highest concentrations of people on weekdays, weekends, etc. With the data given, you can find the most concentrated areas and the days of the week they're most concentrated on. However, you can't (easily) look at how that relates to, for instance, the train lines served at that station, or whether there are unique riders. The primary, most important piece of information I'd look for would be the station locations and to be able to break them down by which line they're on. You can deduce that from this.

Anyone who knows Chicago pretty well probably could have guessed the answers to the two warm-ups. Although it's good to confirm that using data, hopefully we can identify some areas where data will help us discover new things. I'd like to see visualizations of the patterns in the data over time, broken down across all the dimensions we currently have with the two given data sources - day of week, month/season, over the years, train line.    also like to see where data is actually 

One major limitation of this data is that you don't have destinations - you don't know which train people boarded once in the station (train line or direction), or where they got off. Although it would certainly be interesting (and potentially useful) to see the paths of individuals, the major drawback is that you don't know anything about the traffic coming off trains at the station, which could be the majority of traffic at a given station. Since the CTA doesn't require you to tap out when leaving the station, they don't have data on individual riders, but they might have data on turnstiles, so you could at least know where the traffic is concentrated coming out.
 
If you're just looking at where to locate your business, it might not matter to you so much whether people are coming or going (depending on the business). However, you do want an accurate estimate of traffic, and only having the incoming card scans introduces some bias.


One case in which it could be helpful to have only the incoming passengers is if you have a business that's *inside* the train station (i.e. after tapping in), so you have a "captive audience" once they've tapped in but are waiting for their train. This could be something you drop off and then pick up on your way back from work, for example, a la drycleaning. 

## Additional data 
Right off the bat, here are a few types of information that would be very useful to have:
- Passengers getting off trains
- Train lines served by each station
- Locations of stations (and, given that, what types of businesses are nearby) 
- More granular time-of-day ride info  

The first thing we'd want to do is get the locations of all the stations, since this is an inherently spatial problem. Luckily the data portal has [just that!](https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme) At first glance, since I didn't see  the IDs are wonky, I thought we had to join on `stationname`... see that digression below. 

However, there is a field (`map_id`) in the second dataset that can be used to join the two together. When we join them, we get almost twice as many rows, so I coerced them to be one row per station-date pair. In "real life," I'd spend more time "kicking the tires" to make sure this didn't introduce any weird bugs, but that being said, I am all for getting through the whole process once rather than over-engineering on the front end.   

```
create table cta.ridership_with_locations
as (

with ridership as (
select station_id, 
stationname, 
date, 
rides,
daytype
from cta.daily_ridership_by_station
),

station_info as (
select station_descriptive_name,
map_id as station_id,
ada, red, blue, green, brown, purple,
purple_express, pink, yellow, orange,
location 
from cta.station_locations
)

select station_id,
stationname, 
date,
max(rides) as max_rides,
bool_or(ada) as ada, 
bool_or(red) as red, 
bool_or(blue) as blue, 
bool_or(green) as green, 
bool_or(brown) as brown,
bool_or(purple) as purple,
bool_or(purple_express) as purple_express, 
bool_or(pink) as pink, 
bool_or(yellow) as yellow, 
bool_or(orange) as orange,
max(daytype) as daytype,
max(location) as location
from ridership
left join station_info
using (station_id)
group by 1,2,3
);

```
Then we get something that looks like the following:
```
select * from cta.ridership_with_locations limit 10;
 station_id |     stationname      |    date    | max_rides | ada | red | blue | green | brown | purple | purple_express | pink | yellow | orange |        location         
------------+----------------------+------------+-----------+-----+-----+------+-------+-------+--------+----------------+------+--------+--------+-------------------------
      40160 | LaSalle/Van Buren    | 2010-03-16 |      3121 | f   | f   | f    | f     | t     | f      | t              | t    | f      | t      | (41.8768, -87.631739)
      40520 | Foster               | 2002-03-10 |       218 | f   | f   | f    | f     | f     | t      | t              | f    | f      | f      | (42.05416, -87.68356)
      41200 | Argyle               | 2004-04-05 |      2376 | f   | t   | f    | f     | f     | f      | f              | f    | f      | f      | (41.973453, -87.65853)
      ```

This type of granularity is very nice and is in an easy format to create visualizations on a lot of different dimensions. 

## Results Presentation

I have a lot of love for Tableau - it is easy for exploration, makes plots that look really nice, and has the added bonus of plotting geographic data without fussing with PostGIS. A lot of projects that I've worked on, similarly to this one, there is a spatial component, but the necessary spatial *calculations* are not as complicated.

So I created a Tableau notebook and started making some of the visualizations. Specifically, in the exploratory phase, I'd like to investigate trends over time and space in as many dimensions as possible. Is there seasonality in the average ridership of all the train lines? Is this different among the red/blue/purple/green/orange/pink lines? This kidn of exploration is helpful to do while the problem is still open-ended, but even with a very specific problem, you want to explore all of these things to get a good handle on the environment you're working in. As we start to look more closely at the data, this can feed conversations in the real world.

This also feels like a natural place to bring up modeling. Since there is also an inherent temporal component to this problem, it seems like a natural next step to think of some predictive/forecasting model. However, it usually isn't a *great* use of resources to try to model with extreme accuracy the exact number of passengers who will enter at each stop tomorrow (or the next week, month, etc.), because that's *not* very actionable. One way that might be useful is if you have a portable business, like a food truck, you might want to know where to locate in the near future. Then a good way to set up the problem is to try to predict the top 1 (or 3, or 5) stations with the most traffic within the time window you're looking at (within some constraints, e.g. neighborhood, or how close you're allowed to park your food truck to the station). Of course, that would also call for more granular time-level data.

## A long diversion...
Joining them together: 
```
with stations as (select distinct stationname from cta.daily_ridership_by_station) select count(*) from stations left join cta.station_locations on (stations.stationname = cta.station_locations.station_name) where cta.station_locations.station_name is null;
 count 
-------
    77
(1 row)
```
So about half of them don't match right off the bat, and that's because the names in the second dataset have a direction associated with them. If we match on the first 6 characters of the name, that number of unmatched stations for which we have ridership data but no location goes down to 17. For 5, goes down to 12. At some point we're going to get too many fake matches. If we look at the matches using 6: 
```
with stations as (select distinct stationname from cta.daily_ridership_by_station) 

select stationname, station_name from stations 
left join cta.station_locations 
on (substring(stations.stationname from 1 for 5)= substring(cta.station_locations.station_name from 1 for 5));
```

But then for instance we get false matches like Cermak-McCormick Place = Cermak-Chinatown, or can't differentiate between Chicago & State and Chicago & Milwaukee, which are different. 

We need to resolve the names. This seems like it should be a good candidate for regexes - we could match everything from the lefthand table that has "O'hare" and everything from the righthand table that has "O'hare Branch" on their first, say, 7 characters. 

There are three fields that could be used: `stop_name`, `station_name`, and `station_descriptive_name`. But, for example, `station_name` could be `California` for both the California Blue Line stop and the California Green Line stop, although they are not the same physical stop. 

This turns into a really fun entity resolution problem, mostly as an artifact of the grid system in Chicago. Stops that only occur on one train line (e.g. O'Hare Airport, King Drive) don't need any qualifiers. Look at the endings of the stations in the ridership totals: "-Orange", "-Brown", sure, those make sense. "-Cermak" is how many of the Pink Line stop names end, because the Pink Line goes to Cermak or to the Loop. Then it starts to get a little dicier. Blue Line stops (if they have a qualifier) end in "-O'Hare" OR "Forest Park," but unlike in the other dataset, this doesn't denote which direction the train is traveling - it denotes which branch (north or south) of the Blue Line you can find this stop. This matters because there are TWO Harlem stops on the Blue Line. (The Kedzie stop on the Orange Line also runs into 

The last funny thing is that Red Line stops end in "-North Main," rather than "-Red," I suppose because it is alternatively called the "North-South Line." 

AND Washington/Wabash stop doesn't exist in the first dataset because it only opened in 2017! 

```
alter table cta.station_locations add column station_id int;
```
All the stops in the loop can be autolabeled with their unique Id from the original table.
```
update cta.station_locations set station_id = '40680' where station_name = 'Adams/Wabash';
---Washington/Wabash doesn't exist in ridership table
update cta.station_locations set station_id = '40260' where station_name = 'State/Lake';
update cta.station_locations set station_id = '40380' where station_name = 'Clark/Lake';
update cta.station_locations set station_id = '40730' where station_name = 'Washington/Wells';
update cta.station_locations set station_id = '40040' where station_name = 'Quincy/Wells';
update cta.station_locations set station_id = '40160' where station_name = 'LaSalle/Van Buren';
update cta.station_locations set station_id = '40850' where station_name = 'Harold Washington Library-State/Van Buren';
``` 